In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Loading the datasets from the local

In [2]:
# restaurants_limited_features
df_res = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")

# restaurant ids dataset
res_ids = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants.csv")

# yelp general business dataset
business_json_path = "/Users/dima/code/Dimasaur/scorecast/data/yelp_academic_dataset_business.json"

#reading the json file
df_business_raw = pd.read_json(business_json_path, lines=True)

# creating a destination path
csv_file_path_review = "yelp_business_raw.csv"

#converting the output into a csv file
df_business_raw.to_csv(csv_file_path_review, index=False)

/var/folders/fd/585jf60x39vflmpy4gfz7yzh0000gn/T/ipykernel_52812/3914361222.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")


In [3]:
df_res.head()

,Unnamed: 0,index,business_id,postal_code,latitude,longitude,stars,review_count,is_open,food_type,...,good_for_kids,good_for_groups,happy_hour,tv,outdoor_seating,price_range,reservations,table_service,take_out,wheelchair
0,0,3,MTSW4McQd7CbVtyjqoe9mw,19107,39.955505,-75.155564,4.0,80,1,['O'],...,False,False,False,False,False,1,False,False,True,False
1,1,5,CF33F8-E6oudUQ46HnavjQ,37015,36.269593,-87.058943,2.0,6,1,['O'],...,True,True,False,True,True,1,False,False,True,True
2,2,8,k0hlBqXX-Bt0vf1op7Jr1w,63123,38.565165,-90.321087,3.0,19,0,['O'],...,True,True,False,True,True,1,False,False,True,False
3,3,9,bBDDEgkFA1Otx9Lfe7BZUQ,37207,36.208102,-86.768170,1.5,10,1,['O'],...,True,False,False,True,True,1,False,False,True,True
4,4,11,eEOYSgkmpB90uNA7lDOMRA,33602,27.955269,-82.456320,4.0,10,1,['O'],...,False,False,False,False,NaN,False,False,False,False,False


In [4]:
# replace the syntax of the food_type

df_res['food_type_string'] = df_res['food_type'].apply(lambda x: str(x).strip('[]'))
df_res['food_type_string'] = df_res['food_type_string'].apply(lambda x: str(x).replace("'",""))

In [5]:
# check the value counts of the food types
df_res.food_type_string.value_counts(normalize=True).head(20)

food_type_string
O                         0.624570
Pizza                     0.033768
Mexican                   0.027416
Chinese                   0.026020
Burgers, Fast Food        0.015937
Italian, Pizza            0.012972
Italian                   0.011843
American (New)            0.010293
American (Traditional)    0.009700
Fast Food, Sandwiches     0.009700
Japanese, Sushi Bars      0.006275
                          0.006046
Thai                      0.005606
Sandwiches                0.005491
Barbeque                  0.005453
Vietnamese                0.005204
Seafood                   0.005108
Diners                    0.005089
Coffee & Tea, Food        0.004458
Indian                    0.004420
Name: proportion, dtype: float64

In [6]:
# replace the values in the food_types with the top - 20 cuisines

replace_dictionary = {
    "Pizza" : "Italian",
    "Burgers, Fast Food" : "Burgers",
    "Italian, Pizza" : "Italian",
    "American (New)" : "American",
    "Fast Food, Sandwiches" : "Sandwiches",
    "American (Traditional)" : "American",
    "Fast Food, Mexican" : "Mexican",
    "Coffee & Tea, Food" : "Cafes",
    "Japanese, Sushi Bars" : "Japanese",
    "American (Traditional), Diners" : "Diners",
    "Bakeries, Food" : "Bakery",
    "Delis, Sandwiches" : "Delis",
    "Mexican, Tex-Mex" : "Mexican",
    "American (Traditional), Burgers" : "American",
    "Sushi Bars" : "Japanese",
    "Chinese, Fast Food" : "Chinese",
    "American (Traditional), Cafes" : "American",
    "Mexican, Tacos" : "Mexican",
    "Pizza, Sandwiches" : "Sandwiches",
    "Food, Pizza" : "Italian",
    "American (Traditional), Seafood" : "American",
    "Greek, Mediterranean" : "Greek",
    "American (New), American (Traditional)" : "American",
    "Cajun/Creole, Seafood" : "Cajun/Creole"

}

df_res.food_type_string = df_res.food_type_string.replace(replace_dictionary)

# generate the clea array of the top 20 categories
df_food_t20 = list(df_res.food_type_string.value_counts(normalize=True).head(25).index)
df_food_t20.remove('O')
df_food_t20.remove('')
df_food_t20 = df_food_t20[:20]

# top 20 food types
np_food_20 = np.array([df_food_t20])

print(np_food_20)


[['Italian' 'Mexican' 'Chinese' 'American' 'Burgers' 'Sandwiches'
  'Japanese' 'Diners' 'Cafes' 'Delis' 'Thai' 'Barbeque' 'Vietnamese'
  'Seafood' 'Indian' 'Greek' 'Cajun/Creole' 'Bakery' 'Steakhouses'
  'Chicken Wings']]


In [7]:
# filtering the the list of business to get the restaurants only
df_restaurants = df_business_raw[df_business_raw.business_id.isin(res_ids.business_id)].reset_index(drop=True)

# sanity check on the shapes after the transformations of the datasets
print(df_restaurants.shape,df_business_raw.shape, res_ids.shape)

(52268, 14) (150346, 14) (52268, 2)


In [8]:
df_restaurants.categories_clean = df_restaurants.categories.str.replace(", ",",")

/var/folders/fd/585jf60x39vflmpy4gfz7yzh0000gn/T/ipykernel_52812/2457883086.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_restaurants.categories_clean = df_restaurants.categories.str.replace(", ",",")


In [9]:
df_food_t20

['Italian',
 'Mexican',
 'Chinese',
 'American',
 'Burgers',
 'Sandwiches',
 'Japanese',
 'Diners',
 'Cafes',
 'Delis',
 'Thai',
 'Barbeque',
 'Vietnamese',
 'Seafood',
 'Indian',
 'Greek',
 'Cajun/Creole',
 'Bakery',
 'Steakhouses',
 'Chicken Wings']

In [10]:
df_restaurants['food_type_one'] = None

# function that extracts a category from a column in a dataframe if that category is present in a given list


def extract_category(categories, food_types):

    # Combine the lists after splitting by space and by comma
    possible_foods = categories.split(' ') + categories.split(',')
    # Remove any empty strings that might be created by split
    possible_foods = [food.strip() for food in possible_foods if food.strip()]

    for food in possible_foods:
        if food in food_types:
            return food
    return None

df_restaurants['food_type_one'] = df_restaurants['categories'].apply(lambda x: extract_category(x, df_food_t20))

In [11]:
df_restaurants.groupby('food_type_one').stars.mean()

food_type_one
American         3.496080
Barbeque         3.795407
Burgers          2.643650
Cafes            3.988316
Cajun/Creole     3.730947
Chicken Wings    2.931526
Chinese          3.350174
Delis            3.674771
Diners           3.474074
Greek            3.842967
Indian           3.817763
Italian          3.513699
Japanese         3.740715
Mexican          3.494538
Sandwiches       3.471387
Seafood          3.723923
Steakhouses      3.444559
Thai             3.905207
Vietnamese       3.834365
Name: stars, dtype: float64

In [21]:
df_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,food_type_one_Indian,food_type_one_Italian,food_type_one_Japanese,food_type_one_Mexican,food_type_one_Sandwiches,food_type_one_Seafood,food_type_one_Steakhouses,food_type_one_Thai,food_type_one_Vietnamese,food_type_one_None
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
df_restaurants.food_type_one.value_counts(normalize=True).head(20)

food_type_one
American         0.294145
Sandwiches       0.113493
Mexican          0.092620
Italian          0.077301
Burgers          0.072077
Chinese          0.061018
Cafes            0.039274
Seafood          0.037674
Japanese         0.033580
Delis            0.033391
Chicken Wings    0.025602
Barbeque         0.022543
Indian           0.017884
Thai             0.017625
Vietnamese       0.015201
Diners           0.012707
Greek            0.012213
Steakhouses      0.011460
Cajun/Creole     0.010189
Name: proportion, dtype: float64

In [13]:
df_restaurants.food_type_one.isna().sum() / df_restaurants.shape[0]

np.float64(0.1869595163388689)

In [15]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(df_restaurants[['food_type_one']])

df_restaurants[ohe.get_feature_names_out()] = ohe.transform(df_restaurants[['food_type_one']])

df_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,food_type_one_Indian,food_type_one_Italian,food_type_one_Japanese,food_type_one_Mexican,food_type_one_Sandwiches,food_type_one_Seafood,food_type_one_Steakhouses,food_type_one_Thai,food_type_one_Vietnamese,food_type_one_None
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
ohe_columns = ['food_type_one_American', 'food_type_one_Barbeque',
       'food_type_one_Burgers', 'food_type_one_Cafes',
       'food_type_one_Cajun/Creole', 'food_type_one_Chinese',
       'food_type_one_Delis', 'food_type_one_Diners', 'food_type_one_Greek',
       'food_type_one_Indian', 'food_type_one_Italian',
       'food_type_one_Japanese', 'food_type_one_Mexican',
       'food_type_one_Sandwiches', 'food_type_one_Seafood',
       'food_type_one_Steakhouses', 'food_type_one_Thai',
       'food_type_one_Vietnamese', 'food_type_one_None']
ohe_sum = df_restaurants[ohe_columns].sum()
print(ohe_sum)

food_type_one_American        12500.0
food_type_one_Barbeque          958.0
food_type_one_Burgers          3063.0
food_type_one_Cafes            1669.0
food_type_one_Cajun/Creole      433.0
food_type_one_Chinese          2593.0
food_type_one_Delis            1419.0
food_type_one_Diners            540.0
food_type_one_Greek             519.0
food_type_one_Indian            760.0
food_type_one_Italian          3285.0
food_type_one_Japanese         1427.0
food_type_one_Mexican          3936.0
food_type_one_Sandwiches       4823.0
food_type_one_Seafood          1601.0
food_type_one_Steakhouses       487.0
food_type_one_Thai              749.0
food_type_one_Vietnamese        646.0
food_type_one_None             9772.0
dtype: float64


In [20]:
df_restaurants.to_csv('/Users/dima/code/Dimasaur/scorecast/data/restaurants_ohe.csv', index=False)